In [1]:
import numpy as np
import tensorflow as tf

In [15]:
ds_x = tf.data.Dataset.from_tensor_slices([[1.,2],[3,4],[5,6],[7,8],[9,10]])

In [17]:
ds_y = tf.data.Dataset.from_tensor_slices([0,1,2,3,4])

In [18]:
ds_joint = tf.data.Dataset.zip((ds_x, ds_y))

In [19]:
for ex in ds_joint:
    print(ex[0].numpy(), ex[1].numpy())

[1. 2.] 0
[3. 4.] 1
[5. 6.] 2
[7. 8.] 3
[ 9. 10.] 4


In [20]:
ds_map = ds_joint.map(lambda x, y: (x/10.0, y))

In [21]:
for ex in ds_map:
    print(ex[0].numpy(), ex[1].numpy())

[0.1 0.2] 0
[0.3 0.4] 1
[0.5 0.6] 2
[0.7 0.8] 3
[0.9 1. ] 4


In [24]:
tf.random.set_seed(1)
ds = ds_map.shuffle(buffer_size = 100)

In [25]:
for ex in ds:
    print(ex[0].numpy(), ex[1].numpy())

[0.3 0.4] 1
[0.5 0.6] 2
[0.1 0.2] 0
[0.9 1. ] 4
[0.7 0.8] 3


In [26]:
ds_batch = ds.batch(batch_size = 2)

In [27]:
for ex in ds_batch:
    print(ex[0].numpy(), ex[1].numpy())

[[0.1 0.2]
 [0.9 1. ]] [0 4]
[[0.7 0.8]
 [0.5 0.6]] [3 2]
[[0.3 0.4]] [1]


In [28]:
ds_batch2 = ds.batch(batch_size =2, drop_remainder = True)

In [31]:
for ex in ds_batch2:
    print(ex[0].numpy(), ex[1].numpy())

[[0.7 0.8]
 [0.5 0.6]] [3 2]
[[0.3 0.4]
 [0.1 0.2]] [1 0]


In [35]:
import tensorflow_datasets as tfds

In [36]:
iris = tfds.load('iris')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling iris-train.tfrecord...:   0%|          | 0/150 [00:00<?, ? examples/s]

Dataset iris downloaded and prepared to C:\Users\ParkHanna\tensorflow_datasets\iris\2.0.0. Subsequent calls will reuse this data.


In [37]:
type(iris)

dict

In [38]:
iris

{Split('train'): <PrefetchDataset shapes: {features: (4,), label: ()}, types: {features: tf.float32, label: tf.int64}>}

In [39]:
ds = iris['train']

In [40]:
ds = ds.shuffle(buffer_size=150, reshuffle_each_iteration=False)

In [42]:
ds_train = ds.take(100)

In [43]:
ds_test = ds.skip(100)

In [44]:
ds_train = ds_train.map(lambda x: (x['features'], x['label']))
ds_test = ds_test.map(lambda x: (x['features'], x['label']))

In [46]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, input_shape=(4,), activation='sigmoid'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [48]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])

In [49]:
TRAIN_SIZE = 100
NUM_EPOCH = 50
BATCH_SIZE = 2
STEPS_PRE_EPOCH = TRAIN_SIZE / BATCH_SIZE

In [50]:
ds_train = ds_train.shuffle(buffer_size = TRAIN_SIZE).batch(BATCH_SIZE)

In [52]:
model.fit(ds_train, epochs=NUM_EPOCH, steps_per_epoch=STEPS_PRE_EPOCH)

Epoch 1/50
50/50 [==============================] - 1s 2ms/step - loss: 1.0142 - accuracy: 0.3600
Epoch 2/50
50/50 [==============================] - 0s 3ms/step - loss: 0.9476 - accuracy: 0.5300
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8999 - accuracy: 0.8400
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.8551 - accuracy: 0.8600
Epoch 5/50
50/50 [==============================] - 0s 3ms/step - loss: 0.8158 - accuracy: 0.9100
Epoch 6/50
50/50 [==============================] - 0s 4ms/step - loss: 0.7815 - accuracy: 0.8900
Epoch 7/50
50/50 [==============================] - 0s 3ms/step - loss: 0.7473 - accuracy: 0.9100
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.7185 - accuracy: 0.9300
Epoch 9/50
50/50 [==============================] - 0s 3ms/step - loss: 0.6913 - accuracy: 0.9300
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.6666 - accuracy: 0.9500
Epoch 11/50
50/50 [

In [53]:
model.evaluate(ds_test.batch(50))

1/1 [==============================] - 1s 659ms/step - loss: 0.3147 - accuracy: 0.9800


[0.31466591358184814, 0.9800000190734863]